In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/arabicdeepfakedetection5.csv')
df.head()

,Unnamed: 0,Inputs,Label
0,34977,برنامج Essentials Pro 9. 0. 2. 1. 6 Portable ي...,0
1,46463,عرب بت - أعلنت اللجنة المنظمة لمهرجان التسوق ا...,0
2,36231,وكانت وزارة الاستثمار قد رفعت أسعار تلك الخدما...,0
3,71,قال وزير الخارجية البريطاني ، ويليام هيج ، الي...,0
4,42859,حجز تشيلسي فرصة التتويج بلقب دوري أبطال أوروبا...,0


In [ ]:
df=df.drop(columns='Unnamed: 0')


In [ ]:
from sklearn.model_selection import train_test_split
text = df['Inputs']
label = df['Label']


X_train, X_val, y_train, y_val = train_test_split(text, label, test_size=0.25, stratify=label)

In [ ]:
def tokenize_data(tokenizer, texts):


  tokenized_texts = []
  for text in texts:
    # Fix the error by removing the `return_tensors='pt'` argument.
    tokens = tokenizer(text, padding=True, truncation=True)['input_ids']
    tokenized_texts.append(tokens)

  return tokenized_texts

In [ ]:

!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification , AutoModel
from keras.layers import Embedding





tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabert")
model = AutoModel.from_pretrained("aubmindlab/bert-base-arabert")
embedding_layer = Embedding(
    input_dim=model.config.vocab_size,
    output_dim=model.config.hidden_size,
    weights=[(model.get_input_embeddings().weight.detach())],
    trainable=False)

In [ ]:
x =tokenize_data(tokenizer, X_train)
x1 =tokenize_data(tokenizer, X_val)
import keras
import transformers
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
x_train=pad_sequences(x)
x_val=pad_sequences(x1)

In [ ]:
from tensorflow.keras.losses import BinaryCrossentropy
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Input
from keras.layers import Bidirectional
model= Sequential()
model.add(embedding_layer)
model.add(LSTM(16))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=10,steps_per_epoch=100,batch_size=32)

Epoch 1/10
100/100 [==============================] - 6s 20ms/step - loss: 0.5146 - accuracy: 0.7581
Epoch 2/10
100/100 [==============================] - 2s 23ms/step - loss: 0.2045 - accuracy: 0.9341
Epoch 3/10
100/100 [==============================] - 2s 23ms/step - loss: 0.1151 - accuracy: 0.9681
Epoch 4/10
100/100 [==============================] - 2s 23ms/step - loss: 0.0917 - accuracy: 0.9728
Epoch 5/10
100/100 [==============================] - 2s 19ms/step - loss: 0.0888 - accuracy: 0.9734
Epoch 6/10
100/100 [==============================] - 2s 16ms/step - loss: 0.0736 - accuracy: 0.9766
Epoch 7/10
100/100 [==============================] - 2s 16ms/step - loss: 0.0740 - accuracy: 0.9812
Epoch 8/10
100/100 [==============================] - 2s 16ms/step - loss: 0.0536 - accuracy: 0.9828
Epoch 9/10
100/100 [==============================] - 2s 16ms/step - loss: 0.0625 - accuracy: 0.9831
Epoch 10/10
100/100 [==============================] - 2s 16ms/step - loss: 0.0503 - accura

In [ ]:
model.evaluate(x_val,y_val,steps=100)

100/100 [==============================] - 2s 19ms/step - loss: 0.0353 - accuracy: 0.9876


[0.03525465354323387, 0.9876046180725098]

In [ ]:
text=' لرياضة تساعد في تعزيز اللياقة البدنية وتحسين الصحة العامة.'
text_p = [text]
x2 = tokenize_data(tokenizer, text_p)
prediction_input = np.array(x2).reshape(-1)
prediction_input=pad_sequences([prediction_input],128)
output = model.predict (prediction_input)
if output > 0.5:
  predicted_class = 'human written'
else:
  predicted_class = 'machine generated'
# Print the predicted class
print('Predicted class:',predicted_class,output)

1/1 [==============================] - 0s 33ms/step
Predicted class: machine generated [[0.42036954]]


In [ ]:
model.save('aubmindlab-bert-base-arabert.h5')